In [1]:
input_path = "../data"
output_path = "../results"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.figure_factory as ff
import scipy.stats
import pymc3 as pm
import arviz as az
import sunode
import sunode.wrappers.as_theano
from pymc3.ode import DifferentialEquation
import theano.tensor as tt
import theano
import datetime
import shelve
from datetime import datetime as dt
import time


# -------- Usage --------#
# covid_obj = COVID_data('US', Population=328.2e6)
# covid_obj.get_dates(data_begin='7/11/20', data_end='7/20/20')
# sir_model = SIR_model(covid_obj)
# likelihood = {'distribution': 'lognormal', 'sigma': 2}
# prior= {'lam': 0.4, 'mu': 1/8, lambda_std', 0.5 'mu_std': 0.5 }
# sir_model.run_SIR_model(n_samples=20, n_tune=10, likelihood=likelihood)
np.random.seed(0)
 
class COVID_data():
 
    def __init__(self, country='US', Population = 328.2e6):
 
        confirmed_cases_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
        self.confirmed_cases = pd.read_csv(confirmed_cases_url, sep=',')
        deaths_url =  'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv'
        self.deaths = pd.read_csv(deaths_url, sep=',')
        path_to_save = ''
 
        # ------------------------- Country for inference -------------------
 
        self.country = country
        self.N = Population   # Population of the country
                         # Germany - 83.7e6
                         # US - 328.2e6
 
    def get_dates(self, data_begin='7/11/20', data_end='7/20/20'):
 
        # ------------------------- Date for inference ----------------------#
        self.data_begin = data_begin  #Take the data until yesterday
        self.data_end = data_end
        self.num_days_to_predict = 14
        confirmed_cases = self.confirmed_cases
        country = self.country
        self.cases_country = confirmed_cases.loc[confirmed_cases["Country/Region"] == country]
        self.cases_obs = np.array(confirmed_cases.loc[confirmed_cases["Country/Region"] == country, data_begin:data_end])[0]
 
        print("------------ Cases for selected period ----------- ",self.cases_obs)
 
        date_data_end = confirmed_cases.loc[confirmed_cases["Country/Region"] == self.country, data_begin:data_end].columns[-1]
        month, day, year = map(int,date_data_end.split('/'))
        date_data_end = datetime.date(year+2000, month, day)
        date_today = date_data_end + datetime.timedelta(days=1)
        print("------------- Cases yesterday ({}): {} and day before yesterday: {} ------------".format(date_data_end.isoformat(), *self.cases_obs[:-3:-1]))
        self.num_days = len(self.cases_obs)
 
        day_before_start = dt.strptime(data_end, '%m/%d/%y') + datetime.timedelta(days=-1)
        day_before_start_cases = np.array(self.cases_country.loc[:, day_before_start.strftime('%-m/%-d/%-y')])
        print("------------ Day before start and cases for that date ------------", day_before_start, day_before_start_cases)
        future_days_begin = dt.strptime(data_end, '%m/%d/%y') + datetime.timedelta(days=1)
        future_days_end = future_days_begin + datetime.timedelta(days=self.num_days_to_predict)
        self.future_days_begin_s = future_days_begin.strftime('%-m/%-d/%-y')
        self.future_days_end_s = future_days_end.strftime('%-m/%-d/%-y')
        print("------------- Future date begin and end -------------",self.future_days_begin_s, self.future_days_end_s)
        self.future_days = np.array(self.cases_country.loc[:, self.future_days_begin_s : self.future_days_end_s])[0]
        print("------------- Future days cases ------------", self.future_days)
 
 
class SIR_model():
 
    def __init__(self, covid_data) :
 
        # ------------------------- Covid_data object -----------------------#
        self.covid_data = covid_data
        # ------------------------- Setup SIR model, but has to be called explicitly to run ------------------------#
        self.setup_SIR_model()
 
    def SIR_non_normalized(self, y, t, p):
        ds = -p[0] * y[0] * y[1] /self.covid_data.N
        di = p[0] * y[0] * y[1] / self.covid_data.N  -  p[1] * y[1]
        return [ds, di]
 
    def setup_SIR_model(self):
        self.time_range = np.arange(0,len(self.covid_data.cases_obs),1)
        self.I0 = self.covid_data.cases_obs[0]
        self.S0 = self.covid_data.N - self.I0
 
        # SIR model
        self.sir_model_non_normalized = DifferentialEquation(
            func=self.SIR_non_normalized,
            times=self.time_range[1:],
            n_states=2,
            n_theta=2,
            t0=0)
 
    def run_SIR_model(self, n_samples, n_tune, likelihood, prior):
        # ------------------------- Metadata --------------------------------#
        now = dt.now()
        timenow = now.strftime("%d-%m-%Y_%H:%M:%S")
        self.filename = 'sir_' + self.covid_data.data_begin.replace('/','-') + '_' + \
            self.covid_data.data_end.replace('/','-') + '_' + timenow
        self.likelihood = likelihood
        self.n_samples = n_samples
        self.n_tune = n_tune
        self.likelihood = likelihood
        self.prior = prior
        # ------------------------ Write out metadata while the model is running -------------------#
        metadata_db_filename = 'metadata_db.db'
 
        t = time.time()
 
        with pm.Model() as model4:
            sigma = pm.HalfCauchy('sigma', likelihood['sigma'], shape=1)
            lam = pm.Lognormal('lambda', prior['lam'], prior['lambda_std'])
            mu = pm.Lognormal('mu', prior['mu'], prior['mu_std'])
 
            res = self.sir_model_non_normalized(y0=[self.S0, self.I0], theta=[lam, mu])
 
            if(likelihood['distribution'] == 'lognormal'):
                Y = pm.Lognormal('Y', mu=pm.math.log(res.take(0, axis=1)), sigma=sigma, observed=self.covid_data.cases_obs[1:])
            else:
                Y = pm.StudentT( "Y",  nu=likelihood['nu'],       # likelihood distribution of the data
                        mu=res.take(0, axis=1),     # likelihood distribution mean, these are the predictions from SIR
                        sigma=sigma,
                        observed=cases_obs[1:]
                        )
            trace = pm.sample(self.n_samples, tune=self.n_tune, target_accept=0.9, cores=2)
            data = az.from_pymc3(trace=trace)
 
        t1 = time.time() - t
        az.plot_posterior(data, round_to=2, credible_interval=0.95)
        axes = az.plot_trace(trace)
        fig = axes.ravel()[0].figure
        fig.savefig(self.filename)
 
        self.metadata_db = shelve.open(metadata_db_filename)
        self.metadata_db[self.filename] = {'type': 'sir', 'samples': n_samples,
                                    'tune': n_tune,
                                    'elapsed_time': t1,
                                    'finished': dt.now().strftime("%d-%m-%Y_%H:%M:%S"),
                                    'likelihood': likelihood,
                                    'prior': prior }
        self.metadata_db.close()
 
 
 
class SIR_model_sunode():
 
    def __init__(self, covid_data) :
 
        # ------------------------- Covid_data object -----------------------#
        self.covid_data = covid_data
        # ------------------------- Setup SIR model, but has to be called explicitly to run ------------------------#
        self.setup_SIR_model()
 
    def SIR_sunode(self, t, y, p):
        return {
            'S': -p.lam * y.S * y.I,
            'I': p.lam * y.S * y.I - p.mu * y.I,
        }
 
    def setup_SIR_model(self):
        self.time_range = np.arange(0,len(self.covid_data.cases_obs),1)
        self.I0 = self.covid_data.cases_obs[0]
        self.S0 = self.covid_data.N - self.I0
        self.S_init = self.S0 / self.covid_data.N
        self.I_init = self.I0 / self.covid_data.N
        self.cases_obs_scaled = self.covid_data.cases_obs / self.covid_data.N
 
 
    def run_SIR_model(self, n_samples, n_tune, likelihood, prior):
        # ------------------------- Metadata --------------------------------#
        now = dt.now()
        timenow = now.strftime("%d-%m-%Y_%H:%M:%S")
        self.filename = 'sir_' + self.covid_data.data_begin.replace('/','-') + '_' + \
            self.covid_data.data_end.replace('/','-') + '_' + timenow
        self.likelihood = likelihood
        self.n_samples = n_samples
        self.n_tune = n_tune
        self.likelihood = likelihood
        self.prior = prior
        # ------------------------ Write out metadata while the model is running -------------------#
        metadata_db_filename = 'metadata_db.db'
 
        t = time.time()
 
        with pm.Model() as model4:
            sigma = pm.HalfCauchy('sigma', self.likelihood['sigma'], shape=1)
            lam_mu = np.log(self.prior['lam']) + self.prior['lambda_std']**2
            mu_mu = np.log(self.prior['mu']) + self.prior['mu_std']**2
            lam = pm.Lognormal('lambda', lam_mu , self.prior['lambda_std']) # 1.5, 1.5
            mu = pm.Lognormal('mu', mu_mu, self.prior['mu_std'])           # 1.5, 1.5
 
            res, _, problem, solver, _, _ = sunode.wrappers.as_theano.solve_ivp(
            y0={
            # The initial conditions of the ode. Each variable
            # needs to specify a theano or numpy variable and a shape.
            # This dict can be nested.
                'S': (self.S_init, ()),
                'I': (self.I_init, ()),},
            params={
            # Each parameter of the ode. sunode will only compute derivatives
            # with respect to theano variables. The shape needs to be specified
            # as well. It it infered automatically for numpy variables.
            # This dict can be nested.
                'lam': (lam, ()),
                'mu': (mu, ()),
                '_dummy': (np.array(1.), ())},
            # A functions that computes the right-hand-side of the ode using
            # sympy variables.
            rhs=self.SIR_sunode,
            # The time points where we want to access the solution
            tvals=self.time_range,
            t0=self.time_range[0]
            )
            if(likelihood['distribution'] == 'lognormal'):
                I = pm.Lognormal('I', mu=res['I'], sigma=sigma, observed=self.cases_obs_scaled)
            elif(likelihood['distribution'] == 'normal'):
                I = pm.Normal('I', mu=res['I'], sigma=sigma, observed=self.cases_obs_scaled)
            elif(likelihood['distribution'] == 'students-t'):
                I = pm.StudentT( "I",  nu=likelihood['nu'],       # likelihood distribution of the data
                        mu=res['I'],     # likelihood distribution mean, these are the predictions from SIR
                        sigma=sigma,
                        observed=self.cases_obs_scaled
                        )
 
            theano.printing.Print('S')(res['S'])
            print('Problem',problem)
            print('Solver',solver)
 
            R = 1 - (res['I'] + res['S'])
            #S = 1 - (res['I'][1:])
            #theano.printing.Print('R')(R)
            R0 = pm.Deterministic('R0',lam/mu)
 
            step = pm.Metropolis()
            trace = pm.sample(self.n_samples, tune=self.n_tune, chains=4, cores=4)
            data = az.from_pymc3(trace=trace)
 
        t1 = time.time() - t
        az.plot_posterior(data, round_to=2, point_estimate='mode', credible_interval=0.95)
        axes = az.plot_trace(trace)
        fig = axes.ravel()[0].figure
        fig.savefig(self.filename)
        
        fig = ff.create_distplot([trace['R0']], bin_size=0.5, group_labels=['x'])
 
        # Add title
        fig.update_layout(title_text='Curve and Rug Plot')
        fig.update_xaxes(range=[0,7])
              
 
        self.metadata_db = shelve.open(metadata_db_filename)
        self.metadata_db[self.filename] = {'type': 'sir', 'samples': n_samples,
                                    'tune': n_tune,
                                    'elapsed_time': t1,
                                    'finished': dt.now().strftime("%d-%m-%Y_%H:%M:%S"),
                                    'likelihood': likelihood,
                                    'prior': prior }
        self.metadata_db.close()
        return(fig)
 

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


ModuleNotFoundError: No module named 'sunode'

In [ ]:
import numpy as np
from scipy.integrate import odeint
from scipy.optimize import minimize

SEIRS ODE model
def SEIRS(y, t, beta, sigma, gamma, mu):
S, E, I, R, D = y
N = S + E + I + R + D
dSdt = -beta * S * I / N
dEdt = beta * S * I / N - sigma * E
dIdt = sigma * E - gamma * I - mu * I
dRdt = gamma * I
dDdt = mu * I
return [dSdt, dEdt, dIdt, dRdt, dDdt]

optimization function to find the optimal values of beta, sigma, gamma, and mu
def optimize_SEIRS(params, y0, t, data):
# run the ODE model
result = odeint(SEIRS, y0, t, args=params)
# calculate the error between the model results and the data
error = np.sum((result - data)**2)
return error

initial values for the ODE model
y0 = [S0, E0, I0, R0, D0]

time points for the ODE model
t = np.linspace(0, T, num_points)

data for confirmed cases in multiple regions
data = [region1_cases, region2_cases, ...]

bounds for beta, sigma, gamma, and mu
bounds = [(0, 1), (0, 1), (0, 1), (0, 1)]

initial values for beta, sigma, gamma, and mu
params0 = [beta0, sigma0, gamma0, mu0]

minimize the error between the ODE model and the data using metaheuristics
result = minimize(optimize_SEIRS, params0, args=(y0, t, data), bounds=bounds, method='metaheuristics')

print the optimal values for beta, sigma, gamma, and mu
print(result.x)

Consider a country that is currently facing a pandemic. The country is divided into different regions, and each region has a certain population that is in need of a vaccine. The government wants to allocate the available vaccines among the regions in a way that maximizes the number of people who are protected from the disease.

At each period during the pandemic, the government receives a certain number of vaccines that it can distribute among the regions. The goal is to find an optimal allocation of vaccines that maximizes the number of people who are protected from the disease at each period.

The optimization problem can be formulated as follows:

Maximize:
$\sum_{t=1}^{T} \sum_{i=1}^{N} x_{i,t}$

Subject to:

$\sum_{i=1}^{N} x_{i,t} \leq V_t \quad \forall t \in [1, T]$

$x_{i,t} \leq P_{i,t} \quad \forall i \in [1, N], \forall t \in [1, T]$

$x_{i,t} \geq 0 \quad \forall i \in [1, N], \forall t \in [1, T]$

where:

$T$ is the total number of periods during the pandemic.

$N$ is the number of regions in the country.

$V_t$ is the number of vaccines available at period $t$.

$P_{i,t}$ is the population of region $i$ at period $t$.

$x_{i,t}$ is the number of vaccines that are allocated to region $i$ at period $t$.

The objective function seeks to maximize the total number of people who are protected from the disease by allocating the available vaccines among the regions. The first constraint ensures that the number of vaccines allocated at each period does not exceed the number of vaccines available at that period. The second constraint ensures that the number of vaccines allocated to each region at each period does not exceed the population of that region. The final constraint ensures that the number of vaccines allocated to each region at each period is non-negative.

Let's consider a country with N regions and T periods during a pandemic. The objective is to allocate the available vaccine doses among the regions in each period in a way that minimizes the total number of infected individuals while ensuring that each region receives at least a minimum required number of doses.

The decision variables of the problem are:

$x[i, j, t]$ : the number of vaccine doses allocated to region $i$ in period $t$.
The constraints of the problem are:

For each region $i$, the total number of doses allocated in all periods should be at least the minimum required number of doses:

$sum(x[i, j, t] \forall j, t) >= minimumdoses[i] \quad \forall i$
For each period t, the total number of doses allocated should be equal to the total number of available doses:

sum(x[i, j, t] for all i, j) = total_doses[t] for all t
For each region i, the number of doses allocated in each period should be non-negative:

x[i, j, t] >= 0 for all i, j, t
The objective function to minimize is:

sum(infection_rate[i, j] * x[i, j, t] for all i, j, t)
where infection_rate[i, j] is the expected number of infected individuals in region i if it receives j vaccine doses.

The vaccine resource allocation optimization problem for COVID-19 between different regions is to determine the optimal allocation of vaccine doses among the regions at multiple periods during a pandemic in order to maximize the overall population immunity and minimize the spread of the virus.

The decision variables in this problem are the number of vaccine doses allocated to each region at each period. The objective function is to maximize the overall population immunity, which is measured by the total number of vaccinated individuals in all the regions.

The constraints in this problem include the following:

The total number of vaccine doses allocated to each region at each period must be less than or equal to the total number of vaccine doses available at that period.
The total number of vaccine doses allocated to all the regions at each period must be less than or equal to the total number of vaccine doses available at that period.
The number of vaccine doses allocated to each region at each period must be greater than or equal to the minimum number of vaccine doses required to achieve the desired population immunity in that region.
The optimization problem can be formulated as follows:

Maximize: Total number of vaccinated individuals in all the regions

Subject to:

The total number of vaccine doses allocated to each region at each period must be less than or equal to the total number of vaccine doses available at that period.
The total number of vaccine doses allocated to all the regions at each period must be less than or equal to the total number of vaccine doses available at that period.
The number of vaccine doses allocated to each region at each period must be greater than or equal to the minimum number of vaccine doses required to achieve the desired population immunity in that region.

Objective:

Maximize the proportion of the population that is vaccinated in each region.

Minimize the spread of COVID-19 in each region.

Minimize the total cost of the vaccination program.

Constraints:

The number of vaccines available in each period must be limited.

The number of vaccines that can be administered in each region in each period must be limited by the availability of healthcare workers and facilities.

The vaccination rate in each region must be limited by the susceptibility of the population and the effectiveness of the vaccine.

The spread of COVID-19 in each region must be limited by the infection rate and the number of susceptible individuals in the population.

The vaccination rate and the spread of COVID-19 in each region must be modeled using the ODE SEIRS model, which takes into account reinfection.

Decision variables:

The number of vaccines allocated to each region in each period.

The vaccination rate in each region in each period.

The infection rate in each region in each period.

The proportion of the population that is susceptible to COVID-19 in each region in each period.

The total cost of the vaccination program in each period.

Objective:

Maximize the number of individuals who are immune to COVID-19 in each region in the long-term
Minimize the spread of the virus in each region in the short-term
Decision variables:

x_i,t: The amount of vaccine allocated to region i at time t
Constraints:

The total amount of vaccine available at time t is limited by the production capacity:
$$∑_i x_i,t ≤ V_t for all t$$
The allocation of vaccine must follow the SEIR ODE model dynamics:
$$S_i,t+1 = S_i,t - β_i S_i,t I_i,t$$
$$E_i,t+1 = E_i,t + β_i S_i,t I_i,t - κ_i E_i,t$$
$$I_i,t+1 = I_i,t + κ_i E_i,t - γ_i I_i,t - x_i,t$$
$$R_i,t+1 = R_i,t + γ_i I_i,t + x_i,t$$
The allocation of vaccine must consider the current level of immunity in each region:
$$x_i,t ≤ (1 - R_i,t) * S_i,t for all t$$
Objective function:

Maximize the long-term immunity in each region:
$$∑_i (1 - R_i,T) for all T$$
Minimize the short-term spread of the virus in each region:
$$∑_i I_i,t for all t$$
The final optimization problem can be formulated as:

$$maximize ∑_i (1 - R_i,T) - ∑_i I_i,t$$

subject to:

$$∑_i x_i,t ≤ V_t for all t$$

$$S_i,t+1 = S_i,t - β_i S_i,t I_i,t$$

$$E_i,t+1 = E_i,t + β_i S_i,t I_i,t - κ_i E_i,t$$

$$I_i,t+1 = I_i,t + κ_i E_i,t - γ_i I_i,t - x_i,t$$

$$R_i,t+1 = R_i,t + γ_i I_i,t + x_i,t$$

$$x_i,t ≤ (1 - R_i,t) * S_i,t for all t$$

Objective:

Maximize the total number of individuals vaccinated against COVID-19 in each region over multiple periods.
Minimize the total number of individuals infected with COVID-19 in each region over multiple periods.
Decision variables:

x_{i,j,k} - the number of individuals vaccinated against COVID-19 in region i during period j using vaccine k.
y_{i,j} - the number of individuals infected with COVID-19 in region i during period j.
Constraints:

The total number of individuals vaccinated in each region during each period must be less than or equal to the available supply of vaccines in that region:
sum_{k=1}^{n_v} x_{i,j,k} <= S_{i,j}

The number of individuals infected in each region during each period must be determined by the SEIR ODE model:
y_{i,j} = f(y_{i,j-1}, x_{i,j-1}, p_{i,j})

The number of individuals vaccinated in each region during each period must be greater than or equal to zero:
x_{i,j,k} >= 0

The number of individuals infected in each region during each period must be greater than or equal to zero:
y_{i,j} >= 0

Solve the problem using Julia:

In [ ]:
using JuMP
using Gurobi

m = Model(Gurobi.Optimizer)

Decision variables
@variable(m, x[i=1:n_r, j=1:T, k=1:n_v] >= 0) # number of individuals vaccinated in region i during period j using vaccine k
@variable(m, y[i=1:n_r, j=1:T] >= 0) # number of individuals infected in region i during period j

Objective function
@objective(m, Max, sum(x[i,j,k] for i=1:n_r, j=1:T, k=1:n_v) + sum(y[i,j] for i=1:n_r, j=1:T))

Constraints
for i=1:n_r, j=1:T
# The total number of individuals vaccinated in each region during each period must be less than or equal to the available supply of vaccines in that region
@constraint(m, sum(x[i,j,k] for k=1:n_v) <= S[i,j])

Copy code
# The number of individuals infected in each region during each period must be determined by the SEIR ODE model
@constraint(m, y[i,j] == f(y[i,j-1], x[i,j-1], p[i,j]))
end

Solve the optimization problem
optimize!(m)

Retrieve the optimal solution
x_opt = value.(x)
y_opt = value.(y)

In [ ]:
# Import libraries
from sklearn.gaussian_process import GaussianProcessRegressor
import pandas as pd
import numpy as np

# Load time series data into a dataframe
df = pd.read_csv('confirmed_cases.csv')

# Extract time and confirmed cases
time = df['time'].values
confirmed_cases = df['confirmed_cases'].values

# Initialise and fit Gaussian process regressor
gp = GaussianProcessRegressor()
gp.fit(time.reshape(-1, 1), confirmed_cases)

# Predict normalised confirmed cases
normalised_cases = gp.predict(time.reshape(-1, 1))

# Plot original and normalised data
plt.plot(time, confirmed_cases)
plt.plot(time, normalised_cases)
plt.show()